In [ ]:
#variable set up
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import mpmath

#define constants
kT = 4.11*10**(-21)
n_E = 10 #Binding energy
a = 2*10**(-9) #radius of single protein
ϵ = 80 #relative dielectric constant
ϵo = 8.854*10**(-12) #permittivity in vacuum
e = 1.602*10**(-19) #elementary charge
Z = 2 #the number of particle nearest-neighbors
f = 1 #number of interactions
df=1

#define each term in the old model as well as the full models, with and without beta

def Vol( R, n_c, df):
  R= R*10**(-9)
  vol = -Z/2*(R/a)**(df)*n_E*kT
  y= vol
  return y

def Surface( R, n_c, df):
  R=R*10**(-9)
  surf = + ((f*n_E*kT*df)/(a**(df-1)))*R**(df-1)
  y= surf
  return y

def Ent( R, n_c, df):
  R=R*10**(-9)
  ent =  - kT*np.log10(a**3/(R)**(3))
  y= ent
  return y

def EleOld( R, n_c, df):
  R=R*10**(-9)
  ele = ((n_c**2*e**2)/(8*np.pi*ϵ*ϵo*a**(2*df)))*R**(2*df-1)
  y= ele
  return y

def delta(R,n_c, df):
  R = R*10**(-9)
  vol = -Z/2*(R/a)**(df)*n_E*kT
  surface = + ((f*n_E*kT*df)/(a**(df-1)))*R**(df-1)
  ele = + ((n_c**2*e**2)/(8*np.pi*ϵ*ϵo*a**(2*df)))*R**(2*df-1)
  ent = - kT*np.log10(a**3/(R)**(3))
  y = vol + surface + ele + ent
  return y

def deltabeta(R,n_c, df, beta, Z, f):
  R = R*10**(-9)
  vol = -Z/2*(R/a)**(df)*n_E*kT
  surface = + ((f*n_E*kT*df)/(a**(df-1)))*R**(df-1)
  ele = + beta*((n_c**2*e**2)/(8*np.pi*ϵ*ϵo*a**(2*df)))*R**(2*df-1)
  ent = - kT*np.log10(a**3/(R)**(3))
  y = vol + surface + ele + ent
  return y

x_range=np.arange(1, 1000000, 1)  #creating an x array




In [ ]:
#recreation of Fodera figure

scale = 16. #choose scale, charge, and set the desired df values
s = 10**scale
n_c = .5. #charge from Foderà’s paper

dflist = [3,2.75,2.5, 2.0, 1.75]

for t, val in enumerate(dflist):
    df=val
    if 2.5 <= df <= 3.0:
        Z= 6
        f= 3
      
    if 1.0 <= df < 2.5:
        Z= 2
        f=1
    
    temvar = delta(x_range, n_c, val)*s
    globals()[f"list_{t+1}"] = temvar


all_values =[list_1, list_2, list_3, list_4, list_5]

#Finding the index  on the x axis
min_index=np.argmin(all_values, axis=1)
#print(min_index)

for i in range(1,5):   #used if translating the growth pathways
  min=np.min(all_values[i-1])
  all_index=all_values[i][min_index[i-1]]
  difference=min-all_index
  print(min, all_index, difference,i)
  all_values[i] += difference

figure, axis = plt.subplots(1, 2, sharey=True, sharex = True, figsize=(15, 6))

for i in range(5):
  if i != 0:
    plt.plot(x_range[min_index[i-1]:], all_values[i][min_index[i-1]:] , label='df = ' + str(dflist[i]), linestyle = 'dashed')
  else:
    plt.plot(x_range, all_values[i], label='df ='+ str(dflist[i]), linestyle = 'dashed')



for i in range(5):
  if i != 0:
    plt.plot(x_range[min_index[i-1]:min_index[i]], all_values[i][min_index[i-1]:min_index[i]] , color = 'black')
  else:
    plt.plot(x_range[:min_index[i]], all_values[i][:min_index[i]], label='Aggregation Pathway', color = 'black')


plt.xscale('log')
plt.xlim(10,10000)
plt.ylim(-20,10)
plt.title('Title')
plt.xlabel('Cluster Radius (nm)')
plt.ylabel('deltaF (x10^' + str(scale)+ 'J)')
plt.legend(loc='lower left', frameon=False)



for t, val in enumerate(dflist):
  df=val
  if df >= 2.5:
     Z= 6
     f= 3
  else:
      Z= 2
      f=1
  axis[0].plot(x_range, delta(x_range, n_c, val)*s, label='df = ' + str(dflist[t]))
  axis[0].legend(loc='lower left', frameon=False)
  axis[0].set_title('Untranslated')


In [ ]:
#Creating a function to model anisotropic the aggregation pathways or pwathway with beta

def plot_untranslated_pathway(x_range, delta_func, scale=16, n_c=5, dflist=[]):

    s = 10 ** scale
    all_values = []

    # Compute delta values for each df in the list; the 'if ' wan't working so I tried to force it
    #for the low df
    df = (dflist[0])
    Z=2
    f=1
    lowbeta= 1
    tempvar = delta_func(x_range, n_c, df, lowbeta, Z, f) * s
    all_values.append(tempvar)
  
    #for the high df  
    for t, df in enumerate(dflist[1:]):
        Z=6
        f=3
        beta= (0.0000651598) #insert appropriate beta
        tempvar = delta_func(x_range, n_c, df, beta, Z, f) * s
        all_values.append(tempvar)
        

    # Finding the index on the x-axis
    min_index = np.argmin(all_values, axis=1)

    # Aligning the curves based on minimum values
    for i in range(1, len(all_values)):
        min_val = np.min(all_values[i - 1])
        all_index = all_values[i][min_index[i - 1]]
        #difference = min_val - all_index   #use if translating otherwise keep commented out
        #all_values[i] += difference

    # Create subplots 
    figure, axis = plt.subplots(1, 2, sharey=True, sharex=True, figsize=(15, 6))

# First subplot: Untranslated curves
#for the low df
    val = dflist[0]
    Z= 2
    f = 1 
    lowbeta= 1
    axis[0].plot(x_range, delta_func(x_range, n_c, val, lowbeta, Z, f) * s, label='df = ' + str(val))

#for the high values, adjust beta as needed
    for t, val in enumerate(dflist[1:]):
        Z= 6
        f = 3
        beta = (0.0000651598) #insert appropriate beta
        axis[0].plot(x_range, delta_func(x_range, n_c, val, beta, Z, f) * s, label='df = ' + str(val))
        axis[0].legend(loc='lower right', frameon=False)
        axis[0].set_title('Charge = 5')
        axis[0].set_ylabel(f'deltaF (x10^{scale} J)')

    # Second subplot: Aggregation pathway curves
    for i in range(len(all_values)):
        if i != 0:
            axis[1].plot(x_range[min_index[i - 1]:], all_values[i][min_index[i - 1]:],
                          label='df = ' + str(dflist[i]), linestyle='dashed')
        else:
            axis[1].plot(x_range, all_values[i], label='df =' + str(dflist[i]), linestyle='dashed')

    for i in range(len(all_values)):
        if i > 1:
           axis[1].plot(x_range[min_index[i - 1]:min_index[i]],
                          all_values[i][min_index[i - 1]:min_index[i]], color='black')
        elif i == 1:
           axis[1].plot(x_range[500:min_index[i]],
                          all_values[i][500:min_index[i]], color='black')
        else:
           axis[1].plot(x_range[:500], all_values[i][:500],
                          label='Aggregation Pathway', color='black')
    axis[1].axvline(x=500, color = 'black', ymin =0.71, ymax = 0.83 )
            

    # Making the same plot rules for the second plot
    axis[1].set_xscale('log')
    axis[1].set_xlim(10, 1000000)
    axis[1].set_ylim(-50, 10)
   
    axis[1].set_title('Aggregation Pathway 500nm')
    axis[1].set_xlabel('Cluster Radius (nm)')
    axis[1].set_ylabel(f'deltaF (x10^{scale} J)')
    axis[1].legend(loc='lower right', frameon=False)

#####
    # plotting the inlaid plot which shows the initial barrier
    inset = axis[1].inset_axes([0.12, 0.05, 0.3, 0.3])  # [x, y, width, height]
    small_x_range = x_range[:100]  # First 100 x values
    kt_scale= 20-scale
    for t, val in enumerate(dflist):
        inset.plot(small_x_range, delta_func(small_x_range, n_c, val, beta, Z, f)*s*2.3900573613767*10**kt_scale)
    inset.set_xscale('log')
    inset.set_ylabel('deltaF kT')
    #inset.set_ylabel(f'deltaF (x10^({kt_scale})kT)')
    inset.set_ylim(-10, 150)
    # normally inset.set_ylim(-1, 2)
    inset.set_title("Initial barrier")
    inset.tick_params(axis='both', which='both', labelsize=8)
    
######    
    plt.show()

#input (x_range, scale, n_c, and desired df values)
plot_untranslated_pathway(x_range, deltabeta, scale=11, n_c=5, dflist=[1.213, 3, 2.95, 2.9, 2.85, 2.8, 2)
 


In [ ]:
# Define function using symbolic x
def deltabeta_sym(x, n_c, df, beta):
    R = x * 10**(-9)  # Convert nm to meters
    vol = -Z / 2 * (R / a)**df * n_E * kT
    surface = + ((f * n_E * kT * df) / (a**(df - 1))) * R**(df - 1)
    ele = +((n_c**2 * (1.602*10**(-19))**2) / (8 * sp.pi * ϵ * ϵo * a**(2 * df))) * R**(2 * df - 1)
    ent = - kT * sp.log(a**3 / R**3, 10)  # Use SymPy log with base 10
    return vol + surface + beta * ele + ent


In [ ]:
#additional code 1

Z = 2 # Set for low df ranges 
f = 1 
beta = 1 #beta is 1 for low df ranges

df = sp.Symbol('df')

deltabeta_expr = deltabeta_sym(x, 5 , df, beta) # enter charge here

x_target = 5000 # Desired minimum 

df_solution = sp.nsolve(deltabeta_expr.subs(x, x_target), df, 1.5) # Initial guess for df 

print(str(df)+ f" Solved df: {df_solution}")



In [ ]:
#additional code 2
Z = 6 #set for high df ranges
f = 3

dflist = [ 3.0]. #desired df input
for df in dflist:

    #code to get the minima to 500 by solving for beta
    beta = sp.Symbol('beta')


    # compute symbolic derivative
    deltabeta_expr = deltabeta_sym(x, 5, df, beta)  # enter charge
    derivative = sp.diff(deltabeta_expr, x)

    # solve for beta so that the minimum is at x = 500
    x_target = 500  # Desired minimum
    try:
        beta_solution = sp.nsolve(derivative.subs(x, x_target), beta, 0.05)  # Initial guess for beta
        print(str(df)+ f" Solved Beta: {beta_solution}")

        # confirm the result by checking second derivative
        second_derivative = sp.diff(derivative, x)
        is_minimum = second_derivative.subs({x: x_target, beta: beta_solution}) > 0

        if is_minimum:
            min_value = deltabeta_expr.subs({x: x_target, beta: beta_solution})
            print(f"Confirmed Minimum at x = {x_target}, y = {min_value}")
        else:
            print(f"x = {x_target} is not a minimum.")

    except Exception as e:
        print("Error solving for beta:", e)


In [ ]:
#additional code 3

n_c = 5 #desired charge input

dflist = [ 1.1, 1.2, 1.3]. #desired df inputs
bar_df= []


#code to try and get the minimum at your target and to solve for beta
beta = sp.Symbol('beta')
df = 3

# symbolic derivative
deltabeta_expr = deltabeta_sym(x, n_c, df, beta)  # enter charge
derivative = sp.diff(deltabeta_expr, x)
#print(derivative)

# solve for beta so the the minimum is at 500
x_target = 500  # Desired minimum
try:
    beta_solution = sp.nsolve(derivative.subs(x, x_target), beta, 0.5)  # Initial guess for beta
    print(str(df)+ f" = df, Solved Beta: {beta_solution}" + " Changing Z and F, beta = 1, charge = " + str(n_c))

    # confirm the result by checking second derivative
    second_derivative = sp.diff(derivative, x)
    is_minimum = second_derivative.subs({x: x_target, beta: beta_solution}) > 0

    if is_minimum:
        min_value = deltabeta_expr.subs({x: x_target, beta: beta_solution})
        #print(f"Confirmed Minimum at x = {x_target}, y = {min_value}")
    else:
        print(f"x = {x_target} is not a minimum.")
except Exception as e:
    print("Error solving for beta:", e)
for df in dflist:
    if df >= 2.5: 
        Z= 6
        f= 3
        max_x = None
        max_value = None

        beta= beta_solution
        deltabeta_expr = deltabeta_sym(x, n_c, df, beta_solution) #beta is solved for each df

#OR

        deltabeta_expr = deltabeta_sym(x, n_c, df, 1)  # charge and beta solved for 3

        derivative = sp.diff(deltabeta_expr, x)
        #print(derivative)
        #guessing the x value at the maxima
        x_guess = 3

        max_x = sp.nsolve(derivative, x, x_guess)
        max_value = deltabeta_expr.subs(x, max_x).evalf()
        #print(f"Maximum barrier at x = {max_x}, y = {max_value}")
        #kT_barrier = float(max_x*(2.390057361*10**-13))

        kT_barrier = (max_value/(4.11*10**-21))  
        #kT conversion rate 
        print(str(df) + " barrier in kT = " + str(kT_barrier))
        #if kT_barrier > 100:
            #bar_df.append(df)
    else: 
        Z=2
        f=1
        max_x = None
        max_value = None
        #beta= beta_solution
        deltabeta_expr = deltabeta_sym(x, n_c, df, 1)  #charge and beta of 1 for low df
        derivative = sp.diff(deltabeta_expr, x)
        print(derivative)
        #guessing the x value at the maxima
        x_guess = 3

        max_x = sp.nsolve(derivative, x, x_guess)
        max_value = deltabeta_expr.subs(x, max_x).evalf()
        #print(f"Maximum barrier at x = {max_x}, y = {max_value}")
        #kT_barrier = float(max_x*(2.390057361*10**-13))

        kT_barrier = (max_value/(4.11*10**-21))  
        #kT conversion rate 
        print(str(df) + " barrier in kT = " + str(kT_barrier))
        #if kT_barrier > 100:
           # bar_df.append(df)
